In [ ]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import scipy.io


In [2]:
def thresh_split_chanel(img, thresh1, thresh2):
    gray1 = img[:,:,1]
    mask1 = cv.inRange(gray1, 0, thresh1) #90
    gray2 = img[:,:,2]
    mask2 = cv.inRange(gray2, thresh2, 255) #130
    # combine white part of both images
    mask = cv.bitwise_or(mask1, mask2)
    return mask

In [3]:
def center_crop(img, cut_width=0, cut_height=0):
    width, height = img.shape[1], img.shape[0]
    new_width = width - cut_width
    new_height = height - cut_height
    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    return img[int(top):int(bottom), int(left):int(right)]

In [4]:
def crop_rotation(image, box, height, width):
    

    src_pts = box.astype("float32")
    dst_pts = np.array([[0, height],
                        [0, 0],
                        [width, 0],
                        [width, height]], dtype="float32")

    M = cv.getPerspectiveTransform(src_pts, dst_pts)

    # directly warp the rotated rectangle to get the straightened rectangle
    warped = cv.warpPerspective(image, M, (width, height))
    return warped

In [8]:
def segmentation(img_path, filename,rotation):

    global count
    assert os.path.exists(img_path), "File does not exist"
    output_dir = "/home/linhphuong/Documents/documents/anomaly_detection/data_for_YOLOv9/train/lables/"


    min_height = 30
    min_width = 30
    extend_box = 5
    # Đọc ảnh và chuyển đổi sang grayscale
    img = cv.imread(img_path)
    img = center_crop(img, 300)

    thresh = thresh_split_chanel(img, 95, 130)
    # 85 for train
    # 95 for test


    # Loại bỏ nhiễu và xác định vùng nền chắc chắn
    kernel = np.ones((3, 3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=1)
    sure_bg = cv.dilate(opening, kernel, iterations=1)

    # Xác định vùng chữ chắc chắn
    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    _, sure_fg = cv.threshold(dist_transform, 0.001 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # Xác định vùng không xác định
    unknown = cv.subtract(sure_bg, sure_fg)

    # Nhãn các vùng
    _, markers = cv.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Watershed
    markers = cv.watershed(img, markers)

    # Tăng kích thước các đường viền
    markers = markers.astype(np.uint8)
    kernel_dilate = np.ones((3, 3), np.uint8)
    markers = cv.dilate(markers, kernel_dilate, iterations=1)

    # non_background = img.copy()
    # non_background[markers==1] = 0

    objects = np.logical_and(markers != 1, markers != 255)

    # Find contours in the binary_markers image
    contours, _ = cv.findContours(objects.astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # base_image = non_background
    base_image = img

    for contour in contours:    
        # Tính toán hình chữ nhật có diện tích nhỏ nhất bao quanh contour
        x, y, w, h = cv.boundingRect(contour)
        x1 = max(0, x - extend_box)
        y1 = max(0, y - extend_box)
        x2 = min(base_image.shape[1], x + w + extend_box)
        y2 = min(base_image.shape[0], y + h + extend_box)
        if w >= min_width and h >= min_height and x>0 and y>0 and x+w<base_image.shape[1] and y+h<base_image.shape[0]:
            cv.rectangle(base_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            x_center = (x1 + x2)/2/w
            y_center = (y1 + y2)/2/h
    image_name = f"{filename}.png"
    name = f"{count}.txt"
    cv.imwrite(os.path.join(output_dir, image_name), base_image)
    with open(name,"w") as f:
        f.write("0" + " " + str(x_center) + " "+ str(y_center) + " "+ str(w) + " "+ str(h))
        f.write("\n")
        print(filename)
    count+=1

In [9]:
folder_path = '/home/linhphuong/Documents/documents/anomaly_detection/data_for_YOLOv9/train/images/'
count = 0
for filename in os.listdir(folder_path):
    img_path = os.path.join(folder_path, filename)   
    # Gọi hàm a trên ảnh
    segmentation(img_path, filename,rotation=False)

NameError: name 'f' is not defined

In [ ]:
for i in lis:
    link = "/home/linhphuong/anaconda3/gen_data/yolov7/text/total_texttext/TT_new_train_GT/Train/" + i
    mat = scipy.io.loadmat(link)
    mat = mat['gt']
    name = "/home/linhphuong/anaconda3/gen_data/yolov7/text/total_texttext/txt/" + i[3:len(i)-4] + ".txt"
    im = "/home/linhphuong/anaconda3/gen_data/yolov7/text/total_texttext/totaltext/Images/Train/"  + i[3:len(i)-4] + ".jpg"
    print(im)
    img = cv2.imread(im)
    h, w, _ = img.shape
    with open(name,"w") as f:
        for j in mat:
            x_min = j[1].min()
            x_max = j[1].max()
            y_min = j[3].min()
            y_max = j[3].max()
            x_center = (x_max + x_min)/2/w
            y_center = (y_max + y_min)/2/h
            weight = (x_max - x_min)/w
            height = (y_max - y_min)/h
            f.write("0" + " " + str(x_center) + " "+ str(y_center) + " "+ str(weight) + " "+ str(height))
            f.write("\n")